# Roads post-processing

Merge together roads data, clean up

In [3]:
import os, sys
from datetime import date

import pandas as pd
import geopandas as gpd
import numpy as np
from scipy import stats

import re
from datetime import datetime

# custom functions
import sensitivity_testing as st

### Setup

File paths

In [4]:
geo_pth = r'P:\PAK\GEO'
data_dir = r'../../data'

rast_dir = r'rast_inputs'
vect_in_dir = r'vect_inputs'
vect_out_dir = r'vect_out'

rds_dir = r'roads'
dest_dir = r'destinations'
speed_dir = r'speed'
fric_dir = r'friction'
acc_dir = r'access'
tab_dir = r'tabular'

dif_dir = r'access//difs'
ph_dir = r'access//personhours'

Dates

In [5]:
today = datetime.today().strftime('%y%m%d')

In [6]:
# prop_rds_final.to_csv(os.path.join(data_dir,tab_dir,out_tab_name))
# df_stack.to_csv(os.path.join(data_dir,tabular_dir,out_stacktab_name))

Type

In [58]:
# typ = 'absolute_acc_improvements'
# typ = 'relative_acc_improvements'
typ = 'status_quo'

## Combine

In [59]:
dry_rds = f'processed//roads_{typ}_dry_211215.csv'
msn_rds = f'processed//roads_{typ}_msn_211215.csv'
winter_rds = f'processed//roads_{typ}_winter_211215.csv'

In [60]:
dry_rds = pd.read_csv(os.path.join(data_dir,tab_dir,dry_rds)).set_index('Road_ID')
msn_rds = pd.read_csv(os.path.join(data_dir,tab_dir,msn_rds)).set_index('Road_ID')
winter_rds = pd.read_csv(os.path.join(data_dir,tab_dir,winter_rds)).set_index('Road_ID')

Filter down and combine seasonal values

In [61]:
# identify the access columns
msn_acc_cols = list(msn_rds.filter(regex='^(dry|msn|winter)',axis=1).columns)
winter_acc_cols = list(winter_rds.filter(regex='^(dry|msn|winter)',axis=1).columns)

# separate out the columns
msn_acc_data = msn_rds[msn_acc_cols].copy()
winter_acc_data = winter_rds[winter_acc_cols].copy()

In [62]:
msn_acc_data.head(2)

,msn_District_HQs,msn_Provincial_HQ,msn_education_allboys,msn_education_allgirls,msn_education_boys,msn_education_boys_high,msn_education_boys_middle,msn_education_boys_primary,msn_education_girls,msn_education_girls_high,...,msn_education_primary,msn_health_family,msn_health_pharmacies,msn_health_primary,msn_health_private,msn_health_public,msn_health_secondary,msn_health_tertiary,msn_markets_All,msn_markets_Central
Road_ID,,,,,,,,,,,,,,,,,,,,,
1,1548801.0,8661292.0,248.54393,1047.0623,248.54393,2411.811,58953.72,4792.351,1047.0623,25140.404,...,245.51562,21696.008,13362.949,11455.447,22437.129,46738.580,21782.975,19323544.0,24984.57,14604438.0
2,2253946.0,9447502.0,13645.70800,17699.8710,13645.70800,28784.787,143670.84,14581.795,17699.8710,75989.760,...,11345.96700,75796.670,78226.234,82237.020,96157.120,52965.457,93563.336,21518904.0,3176062.20,36289080.0


In [63]:
winter_acc_data.head(2)

,winter_District_HQs,winter_Provincial_HQ,winter_education_allboys,winter_education_allgirls,winter_education_boys,winter_education_boys_high,winter_education_boys_middle,winter_education_boys_primary,winter_education_girls,winter_education_girls_high,...,winter_education_primary,winter_health_family,winter_health_pharmacies,winter_health_primary,winter_health_private,winter_health_public,winter_health_secondary,winter_health_tertiary,winter_markets_All,winter_markets_Central
Road_ID,,,,,,,,,,,,,,,,,,,,,
1,1457792.2,6321580.5,199.14967,862.1647,199.14967,1006.2777,8195.102,4541.3164,862.1647,22790.432,...,204.79694,16748.123,10680.547,9510.233,18413.307,40383.970,18023.80,17239654.0,18318.197,13127025.0
2,2044643.5,8232845.5,11480.78600,16071.5370,11480.78600,23728.5600,131513.000,12031.0200,16071.5370,80356.960,...,10634.36800,75138.080,76582.840,70542.320,90363.560,53045.227,88219.45,19139530.0,2974328.500,31784074.0


Merge, check

In [65]:
roads_fin = pd.concat([dry_rds,msn_acc_data,winter_acc_data],axis=1,ignore_index=False)
roads_fin

,District,Current_Road_Cond,Current_Surface,Current_Road_Class,Upgrade_Road_Cond,Upgrade_Surface,Upgrade_Road_Class,Terrain,Cost_PKR,current_base_speed,...,winter_education_primary,winter_health_family,winter_health_pharmacies,winter_health_primary,winter_health_private,winter_health_public,winter_health_secondary,winter_health_tertiary,winter_markets_All,winter_markets_Central
Road_ID,,,,,,,,,,,,,,,,,,,,,
1,Upper Chitral,Poor,Earthen,Collector Road,Very Good,Asphaltic,Collector Road,Mountains,5.318627e+08,10,...,204.79694,16748.123,1.068055e+04,9.510233e+03,1.841331e+04,40383.9700,1.802380e+04,17239654.0,18318.197,13127025.0
2,Upper Chitral,Poor,Earthen,Collector Road,Very Good,Asphaltic,Collector Road,Mountains,2.013998e+09,10,...,10634.36800,75138.080,7.658284e+04,7.054232e+04,9.036356e+04,53045.2270,8.821945e+04,19139530.0,2974328.500,31784074.0
3,Upper Chitral,Poor,Earthen,Collector Road,Very Good,Asphaltic,Collector Road,Plains,2.535872e+08,20,...,2338.60900,118110.266,3.270611e+04,9.857966e+03,1.489260e+05,6224.4060,1.133145e+06,11880742.0,149559.420,13134192.0
4,Upper Chitral,Poor,Earthen,Collector Road,Very Good,Asphaltic,Collector Road,Mountains,7.116544e+08,10,...,4151.01030,39190.780,3.183449e+04,3.250495e+04,4.607097e+04,33801.1680,4.464275e+04,11864256.0,44816.273,13127025.0
5,Upper Chitral,Poor,Earthen,Collector Road,Very Good,Asphaltic,Collector Road,Mountains,1.961379e+08,10,...,604.13630,399063.970,2.374995e+03,2.295709e+03,2.352561e+03,2101.2122,2.162198e+03,11864256.0,2171.310,28700922.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,Kurram,Poor,Gravel,District Road,Very Good,Asphaltic,District Road,Plains,2.139936e+08,50,...,2255.37800,81781.850,3.418541e+05,4.421313e+04,1.722732e+06,5371.5300,1.345173e+04,6004329.0,113734.690,6138439.0
84,Kurram,Poor,Gravel,District Road,Very Good,Asphaltic,District Road,Plains,1.355558e+08,50,...,599.64820,63308.620,1.571602e+04,1.684447e+05,4.042865e+04,96277.8750,3.773039e+03,6947170.5,1120438.500,5019684.5
85,North Waziristan,Poor,Gravel,District Road,Very Good,Asphaltic,District Road,Hills,1.658672e+09,40,...,4579.65530,495066.600,8.092501e+05,2.187000e+06,1.486690e+06,227550.9000,2.445313e+05,12006012.0,2144264.200,10571464.0


### Export

In [66]:
roads_fin_pth = os.path.join(data_dir,tab_dir,f"final//roads_{typ}_data_combined_{today}.csv")

In [67]:
roads_fin.to_csv(roads_fin_pth)